# 1D Burgers Exact Solution using Gaussian Quadrature Method (GQM)

*Last edited: 2024-04-26*

Attempt to recreate the exact solution dataset [burgers_shock.mat](https://github.com/maziarraissi/PINNs/blob/master/appendix/Data/burgers_shock.mat) 256x100 used in [Raissi's work](https://github.com/maziarraissi/PINNs). The data uses an n × m grid (e.g., a 256 × 100 grid represents 256 spatial measurements at 100 time points). Other datasets are also generated, 128x128, 256x128, 256x256, 512x256, and 512x512.

Based on Burkardt's Python code: https://people.sc.fsu.edu/~jburkardt/py_src/burgers_solution/burgers_solution.py

As far I can understand, the exact solutions of the Burgers equation use estimated values calculated using [Hermite's quadrature rule](https://en.wikipedia.org/wiki/Gauss%E2%80%93Hermite_quadrature), as described in the [Basdevant et al. article](https://www.academia.edu/19081708/Spectral_and_finite_difference_solutions_of_the_Burgers_equation). 

Using [Burkardt's Python code](https://people.sc.fsu.edu/~jburkardt/py_src/burgers_solution/burgers_solution.html) (see below) with parameters vtn=100, vxn=256, nu=0.01/numpy.pi, xlo=-1.0, xhi=1.0, tlo=0.0, thi=0.99, and qn=50 (order of the quadrature rule), it is possible to generate an array containing exact solutions, which passes the test [numpy.allclose](https://numpy.org/doc/stable/reference/generated/numpy.allclose.html) when compared to the original array of solutions contained in *burgers_shock.mat* from Raissi.

In [1]:
def burgers_solution_test(vxn, vtn, dsname):

    #*****************************************************************************80
    #
    ## burgers_solution_test tests burgers_solution.
    #
    #  Licensing:
    #
    #    This code is distributed under the GNU LGPL license.
    #
    #  Modified:
    #
    #    27 September 2015
    #
    #  Author:
    #
    #    John Burkardt
    #
    import platform

    #print('')
    #print('burgers_solution_test():')
    #print('  Python version: %s' % (platform.python_version()))
    #print('  burgers_solution evaluates exact solutions of the Burgers equation.')

    burgers_viscous_time_exact1_test01(vxn, vtn, dsname)

    #
    #  Terminate.
    #
    #print('')
    #print('burgers_solution_test():')
    #print('  Normal end of execution.')
    return


def burgers_viscous_time_exact1(nu, vxn, vx, vtn, vt):
    """
    Burgers_viscous_time_exact1() evaluates a solution to the Burgers equation.
    
     Discussion:
    
       The form of the Burgers equation considered here is
    
         du       du        d^2 u
         -- + u * -- = nu * -----
         dt       dx        dx^2
    
       for -1.0 < x < +1.0, and 0 < t.
    
       Initial conditions are u(x,0) = - sin(pi*x).  Boundary conditions
       are u(-1,t) = u(+1,t) = 0.  The viscosity parameter nu is taken
       to be 0.01 / pi, although this is not essential.
    
       The authors note an integral representation for the solution u(x,t),
       and present a better version of the formula that is amenable to
       approximation using Hermite quadrature.
    
       This program library does little more than evaluate the exact solution
       at a user-specified set of points, using the quadrature rule.
       Internally, the order of this quadrature rule is set to 8, but the
       user can easily modify this value if greater accuracy is desired.
    
     Licensing:
    
       This code is distributed under the GNU LGPL license.
    
     Modified:
    
       24 September 2015
    
     Author:
    
       John Burkardt.
    
     Reference:
    
       Claude Basdevant, Michel Deville, Pierre Haldenwang, J Lacroix,
       J Ouazzani, Roger Peyret, Paolo Orlandi, Anthony Patera,
       Spectral and finite difference solutions of the Burgers equation,
       Computers and Fluids,
       Volume 14, Number 1, 1986, pages 23-41.
    
     Input:
    
       real NU, the viscosity.
    
       integer VXN, the number of spatial grid points.
    
       real VX(VXN), the spatial grid points.
    
       integer VTN, the number of time grid points.
    
       real VT(VTN), the time grid points.
    
     Output:
    
       real VU(VXN,VTN), the solution of the Burgers
       equation at each space and time grid point.
    """
    
    import numpy as np
    
    #---------------------------------------
    # qn = 8   # original value
    #qn = 50
    print('Quadrature order = %d' % (qn))
    #---------------------------------------
    
    #
    #  Compute the rule.
    #
    qx, qw = hermite_ek_compute(qn)
    #
    #  Evaluate U(X,T) for later times.
    #
    vu = np.zeros([vxn, vtn])

    for vti in range(0, vtn):

        if (vt[vti] == 0.0):

            for i in range(0, vxn):
                vu[i, vti] = -np.sin(np.pi * vx[i])

        else:

            for vxi in range(0, vxn):

                top = 0.0
                bot = 0.0

                for qi in range(0, qn):

                    c = 2.0 * np.sqrt(nu * vt[vti])

                    top = top - qw[qi] * c * np.sin ( np.pi * ( vx[vxi] - c * qx[qi] ) ) \
                      * np.exp ( - np.cos ( np.pi * ( vx[vxi] - c * qx[qi]  ) ) \
                      / ( 2.0 * np.pi * nu ) )

                    bot = bot + qw[qi] * c \
                      * np.exp ( - np.cos ( np.pi * ( vx[vxi] - c * qx[qi]  ) ) \
                      / ( 2.0 * np.pi * nu ) )

                    vu[vxi, vti] = top / bot

    return vu


def burgers_viscous_time_exact1_test01(vxn, vtn, dsname):

    #*****************************************************************************80
    #
    ## burgers_viscous_time_exact1_test01() tests sets up a small test case.
    #
    #  Licensing:
    #
    #    This code is distributed under the GNU LGPL license.
    #
    #  Modified:
    #
    #    24 September 2015
    #
    #  Author:
    #
    #    John Burkardt
    #
    import numpy as np
    import platform

    #vtn = 100
    #vxn = 256
    #nu = 0.01 / np.pi
    #thi = 3.0 / np.pi
    # xlo = -1.0
    # xhi = +1.0
    # tlo = 0.0
    # thi = 0.99

    #print('')
    #print('burgers_viscous_time_exact1_test01():')
    #print('  Python version: %s' % (platform.python_version()))
    #print('  burgers_viscous_time_exact1() evaluates solution #1')
    #print('  to the Burgers equation.')
    #print('')
    print('Viscosity NU = %g' % (nu))
    print('NX = %d' % (vxn))
    print('NT = %d' % (vtn))
    
    vx = np.linspace(xlo, xhi, vxn)
    x_save = vx.flatten()[:, None]
    #r8vec_print(vxn, vx, '  X grid points:')
    #np.savetxt("burgers_solution_x.csv", vx, delimiter=",")

    vt = np.linspace(tlo, thi, vtn)
    t_save = vt.flatten()[:, None]
    #r8vec_print(vtn, vt, '  T grid points:')
    #np.savetxt("burgers_solution_t.csv", vt, delimiter=",")

    vu = burgers_viscous_time_exact1(nu, vxn, vx, vtn, vt)
    #np.savetxt("burgers_solution_u.csv", vu, delimiter=",")

    np.savez_compressed("../data/"+dsname, x=x_save, t=t_save, usol=vu)

    #r8mat_print(vxn, vtn, vu, '  U(X,T) at grid points:')
    #filename = 'burgers_solution_test01.txt'
    #r8mat_write(filename, vxn, vtn, vu)
    #print('')
    #print('  Data written to file "%s"' % (filename))
    #
    #  Terminate
    #
    #print('')
    #print('burgers_viscous_time_exact1_test01():')
    #print('  Normal end of execution.')
    return



def hermite_ek_compute(n):

    #*****************************************************************************80
    #
    ## hermite_ek_compute() computes a Gauss-Hermite quadrature rule.
    #
    #  Discussion:
    #
    #    The code uses an algorithm by Elhay and Kautsky.
    #
    #    The abscissas are the zeros of the N-th order Hermite polynomial.
    #
    #    The integral:
    #
    #      integral ( -oo < x < +oo ) exp ( - x * x ) * f(x) dx
    #
    #    The quadrature rule:
    #
    #      sum ( 1 <= i <= n ) w(i) * f ( x(i) )
    #
    #  Licensing:
    #
    #    This code is distributed under the GNU LGPL license.
    #
    #  Modified:
    #
    #    15 June 2015
    #
    #  Author:
    #
    #    John Burkardt.
    #
    #  Reference:
    #
    #    Sylvan Elhay, Jaroslav Kautsky,
    #    Algorithm 655: IQPACK, FORTRAN Subroutines for the Weights of
    #    Interpolatory Quadrature,
    #    ACM Transactions on Mathematical Software,
    #    Volume 13, Number 4, December 1987, pages 399-415.
    #
    #  Input:
    #
    #    integer N, the number of abscissas.
    #
    #  Output:
    #
    #    real X(N), the abscissas.
    #
    #    real W(N), the weights.
    #
    from scipy.special import gamma
    import numpy as np
    #
    #  Define the zero-th moment.
    #
    zemu = gamma(0.5)
    #
    #  Define the Jacobi matrix.
    #
    bj = np.zeros(n)
    for i in range(0, n):
        bj[i] = np.sqrt(float(i + 1) / 2.0)

    x = np.zeros(n)

    w = np.zeros(n)
    w[0] = np.sqrt(zemu)
    #
    #  Diagonalize the Jacobi matrix.
    #
    x, w = imtqlx(n, x, bj, w)
    #
    #  If N is odd, force the center X to be exactly 0.
    #
    if ((n % 2) == 1):
        x[(n - 1) // 2] = 0.0

    for i in range(0, n):
        w[i] = w[i]**2

    return x, w

In [5]:
def imtqlx(n, d, e, z):
    lam = np.zeros(n)
    for i in range(0, n):
        lam[i] = d[i]
    qtz = np.zeros(n)
    for i in range(0, n):
        qtz[i] = z[i]
    if n == 1:
        return lam, qtz
    itn = 30
    epsilon = np.finfo(float).eps
    e[n - 1] = 0.0
    for l in range(1, n + 1):
        j = 0
        while True:
            for m in range(l, n + 1):
                if m == n:
                    break
                if abs(e[m - 1]) <= epsilon * (abs(lam[m - 1]) + abs(lam[m])):
                    break
            p = lam[l - 1]
            if m == l:
                break
            if itn <= j:
                print("")
                print("imtqlx - Fatal error!")
                print("  Iteration limit exceeded.")
                raise Exception("imtqlx - Fatal error!")
            j = j + 1
            g = (lam[l] - p) / (2.0 * e[l - 1])
            r = np.sqrt(g * g + 1.0)
            if g < 0.0:
                t = g - r
            else:
                t = g + r
            g = lam[m - 1] - p + e[l - 1] / (g + t)
            s = 1.0
            c = 1.0
            p = 0.0
            mml = m - l
            for ii in range(1, mml + 1):
                i = m - ii
                f = s * e[i - 1]
                b = c * e[i - 1]
                if abs(g) <= abs(f):
                    c = g / f
                    r = np.sqrt(c * c + 1.0)
                    e[i] = f * r
                    s = 1.0 / r
                    c = c * s
                else:
                    s = f / g
                    r = np.sqrt(s * s + 1.0)
                    e[i] = g * r
                    c = 1.0 / r
                    s = s * c
                g = lam[i] - p
                r = (lam[i - 1] - g) * s + 2.0 * c * b
                p = s * r
                lam[i] = g + p
                g = c * r - b
                f = qtz[i]
                qtz[i] = s * qtz[i - 1] + c * f
                qtz[i - 1] = c * qtz[i - 1] - s * f
            lam[l - 1] = lam[l - 1] - p
            e[l - 1] = g
            e[m - 1] = 0.0
    for ii in range(2, n + 1):
        i = ii - 1
        k = i
        p = lam[i - 1]
        for j in range(ii, n + 1):
            if lam[j - 1] < p:
                k = j
                p = lam[j - 1]
        if k != i:
            lam[k - 1] = lam[i - 1]
            lam[i - 1] = p
            p = qtz[i - 1]
            qtz[i - 1] = qtz[k - 1]
            qtz[k - 1] = p
    return lam, qtz

In [6]:
import numpy as np
from scipy.io import loadmat

---
## Problem 01

viscosity = 0.01/np.pi

In [3]:
nu = 0.01 / np.pi
xlo = -1.0
xhi = +1.0
tlo = 0.0
thi = 0.99
qn = 50  # quadrature order. control the accuracy.

In [140]:
%%time
burgers_solution_test(256, 100, "burgers256x100")

Viscosity NU = 0.0031831
NX = 256
NT = 100
Quadrature order = 50
CPU times: user 7.82 s, sys: 0 ns, total: 7.82 s
Wall time: 7.82 s


In [17]:
npzfile = np.load("burgers256x100.npz")

In [18]:
npzfile.files

['x', 't', 'usol']

In [19]:
npzfile['x'].size, npzfile['t'].size

(256, 100)

In [60]:
npzfile['x']

array([[-1.        ],
       [-0.99215686],
       [-0.98431373],
       [-0.97647059],
       [-0.96862745],
       [-0.96078431],
       [-0.95294118],
       [-0.94509804],
       [-0.9372549 ],
       [-0.92941176],
       [-0.92156863],
       [-0.91372549],
       [-0.90588235],
       [-0.89803922],
       [-0.89019608],
       [-0.88235294],
       [-0.8745098 ],
       [-0.86666667],
       [-0.85882353],
       [-0.85098039],
       [-0.84313725],
       [-0.83529412],
       [-0.82745098],
       [-0.81960784],
       [-0.81176471],
       [-0.80392157],
       [-0.79607843],
       [-0.78823529],
       [-0.78039216],
       [-0.77254902],
       [-0.76470588],
       [-0.75686275],
       [-0.74901961],
       [-0.74117647],
       [-0.73333333],
       [-0.7254902 ],
       [-0.71764706],
       [-0.70980392],
       [-0.70196078],
       [-0.69411765],
       [-0.68627451],
       [-0.67843137],
       [-0.67058824],
       [-0.6627451 ],
       [-0.65490196],
       [-0

In [23]:
import scipy.io
data = scipy.io.loadmat("../data/burgers_shock.mat")

In [24]:
data['x'].size, data['t'].size, data['usol'].size

(256, 100, 25600)

In [37]:
data['x']

array([[-1.        ],
       [-0.99215686],
       [-0.98431373],
       [-0.97647059],
       [-0.96862745],
       [-0.96078431],
       [-0.95294118],
       [-0.94509804],
       [-0.9372549 ],
       [-0.92941176],
       [-0.92156863],
       [-0.91372549],
       [-0.90588235],
       [-0.89803922],
       [-0.89019608],
       [-0.88235294],
       [-0.8745098 ],
       [-0.86666667],
       [-0.85882353],
       [-0.85098039],
       [-0.84313725],
       [-0.83529412],
       [-0.82745098],
       [-0.81960784],
       [-0.81176471],
       [-0.80392157],
       [-0.79607843],
       [-0.78823529],
       [-0.78039216],
       [-0.77254902],
       [-0.76470588],
       [-0.75686275],
       [-0.74901961],
       [-0.74117647],
       [-0.73333333],
       [-0.7254902 ],
       [-0.71764706],
       [-0.70980392],
       [-0.70196078],
       [-0.69411765],
       [-0.68627451],
       [-0.67843137],
       [-0.67058824],
       [-0.6627451 ],
       [-0.65490196],
       [-0

In [61]:
data['usol']

array([[ 1.22464680e-16,  2.95362215e-17,  1.08470836e-16, ...,
         3.57297976e-16,  2.61833228e-16,  9.39536897e-17],
       [ 2.46374492e-02,  2.38801772e-02,  2.31684474e-02, ...,
         6.07646192e-03,  6.02971754e-03,  5.98368729e-03],
       [ 4.92599411e-02,  4.77471404e-02,  4.63251758e-02, ...,
         1.21528662e-02,  1.20593792e-02,  1.19673204e-02],
       ...,
       [-4.92599411e-02, -4.77471404e-02, -4.63251758e-02, ...,
        -1.21528662e-02, -1.20593792e-02, -1.19673204e-02],
       [-2.46374492e-02, -2.38801772e-02, -2.31684474e-02, ...,
        -6.07646192e-03, -6.02971754e-03, -5.98368729e-03],
       [-1.22464680e-16, -1.08544418e-16, -1.29376253e-16, ...,
         1.04877526e-16,  2.17508692e-16,  1.12388795e-16]])

In [62]:
npzfile['usol']

array([[ 1.22464680e-16,  2.16583317e-16,  1.16508266e-16, ...,
         1.04168654e-16,  7.11914489e-17, -6.49341278e-17],
       [ 2.46374492e-02,  2.38801772e-02,  2.31684474e-02, ...,
         6.07646188e-03,  6.02971749e-03,  5.98368723e-03],
       [ 4.92599411e-02,  4.77471404e-02,  4.63251758e-02, ...,
         1.21528662e-02,  1.20593792e-02,  1.19673204e-02],
       ...,
       [-4.92599411e-02, -4.77471404e-02, -4.63251758e-02, ...,
        -1.21528662e-02, -1.20593792e-02, -1.19673204e-02],
       [-2.46374492e-02, -2.38801772e-02, -2.31684474e-02, ...,
        -6.07646188e-03, -6.02971749e-03, -5.98368723e-03],
       [-1.22464680e-16, -1.77233302e-16, -1.54342007e-16, ...,
        -6.56160778e-17,  6.20360074e-17, -6.64620655e-18]])

In [7]:
%%time
burgers_solution_test(128, 64, "burgers128x64")

Viscosity NU = 0.0031831
NX = 128
NT = 64
Quadrature order = 50
CPU times: user 2.09 s, sys: 0 ns, total: 2.09 s
Wall time: 2.12 s


In [75]:
%%time
burgers_solution_test(128, 128, "burgers128x128")

  Viscosity NU = 0.0031831
  NX = 128
  NT = 128
  Quadrature order = 50
CPU times: user 4.54 s, sys: 3.9 ms, total: 4.54 s
Wall time: 4.54 s


In [80]:
%%time
burgers_solution_test(256, 128, "burgers256x128")

  Viscosity NU = 0.0031831
  NX = 256
  NT = 128
  Quadrature order = 50
CPU times: user 10.4 s, sys: 3.89 ms, total: 10.4 s
Wall time: 10.4 s


In [81]:
%%time
burgers_solution_test(256, 256, "burgers256x256")

  Viscosity NU = 0.0031831
  NX = 256
  NT = 256
  Quadrature order = 50
CPU times: user 21.4 s, sys: 3.87 ms, total: 21.4 s
Wall time: 21.4 s


In [82]:
%%time
burgers_solution_test(512, 256, "burgers512x256")

  Viscosity NU = 0.0031831
  NX = 512
  NT = 256
  Quadrature order = 50
CPU times: user 42.2 s, sys: 11.8 ms, total: 42.2 s
Wall time: 42.2 s


In [83]:
%%time
burgers_solution_test(512, 512, "burgers512x512")

  Viscosity NU = 0.0031831
  NX = 512
  NT = 512
  Quadrature order = 50
CPU times: user 1min 24s, sys: 3.83 ms, total: 1min 24s
Wall time: 1min 25s


---
## Problem 02

viscosity = 0.1/np.pi

In [16]:
nu = 0.1/np.pi
xlo = -1.0
xhi = +1.0
tlo = 0.0
thi = 0.99
qn = 50  # quadrature order

In [12]:
%%time
dsname = "burgers256x100-02.npz"
burgers_solution_test(256, 101, "../data/"+dsname)

Viscosity NU = 0.031831
NX = 256
NT = 101
Quadrature order = 50
CPU times: user 7.37 s, sys: 0 ns, total: 7.37 s
Wall time: 7.37 s


In [134]:
npzfile = np.load("../data/burgers256x100-02.npz")
t = np.ravel(npzfile['t'])  # ravel returns a contiguous flattened array
x = np.ravel(npzfile['x'])
u = np.real(npzfile['usol'])

In [135]:
npzfile.files

['x', 't', 'usol']

In [136]:
npzfile['x'].size, npzfile['t'].size

(256, 101)

In [13]:
%%time
burgers_solution_test(128, 64, "burgers128x64-02")

Viscosity NU = 0.031831
NX = 128
NT = 64
Quadrature order = 50
CPU times: user 2.13 s, sys: 0 ns, total: 2.13 s
Wall time: 2.13 s


In [17]:
%%time
burgers_solution_test(256, 128, "burgers256x128-02")

Viscosity NU = 0.031831
NX = 256
NT = 128
Quadrature order = 50
CPU times: user 9.55 s, sys: 0 ns, total: 9.55 s
Wall time: 9.55 s


In [18]:
%%time
burgers_solution_test(512, 256, "burgers512x256-02")

Viscosity NU = 0.031831
NX = 512
NT = 256
Quadrature order = 50
CPU times: user 41.2 s, sys: 0 ns, total: 41.2 s
Wall time: 41.2 s


---
## Problem 03

viscosity = 0.001/np.pi

In [144]:
0.001/np.pi

0.0003183098861837907

In [145]:
nu = 0.001/np.pi
xlo = -1.0
xhi = +1.0
tlo = 0.0
thi = 0.99
qn = 50  # quadrature order

In [146]:
%%time
dsname = "burgers256x100-03.npz"
burgers_solution_test(256, 101, "../data/"+dsname)

Viscosity NU = 0.00031831
NX = 256
NT = 101
Quadrature order = 50
CPU times: user 7.66 s, sys: 0 ns, total: 7.66 s
Wall time: 7.68 s


---
## References

Raissi, M., Perdikaris, P., & Karniadakis, G. E. (2019). Physics-informed neural networks: A deep learning framework for solving forward and inverse problems involving nonlinear partial differential equations. Journal of Computational Physics, 378, 686–707. https://doi.org/10.1016/j.jcp.2018.10.045

Basdevant, C., Deville, M., Haldenwang, P., Lacroix, J. M., Ouazzani, J., Peyret, R., Orlandi, P., & Patera, A. T. (1986). Spectral and finite difference solutions of the Burgers equation. Computers & Fluids, 14(1), Article 1. https://doi.org/10.1016/0045-7930(86)90036-8

Burkardt, J. (2013). Investigating Uncertain Parameters in the Burgers Equation. Mathematics Department, Ajou University, Suwon, Korea. https://people.sc.fsu.edu/~jburkardt/presentations/burgers_2013_ajou.pdf

<hr style="height:10px;border-width:0;background-color:gray">

## Environment

In [1]:
%%bash
source $HOME/miniconda3/bin/activate psin
conda list --explicit > Burgers-1D-Exact-Solution-environment.txt